In [168]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

In [169]:
data = pd.read_csv('/kaggle/input/digit-recognizer/test.csv')

In [170]:
data.head()

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [171]:
data = np.array(data)
m, n = data.shape
np.random.shuffle(data)

data_dev = data[0: 1000].T
Y_dev = data_dev[0] 
X_dev = data_dev[1:n]

data_train = data[1000:m].T
Y_train = data_train[0] 
X_train = data_train[1:n]

In [172]:
X_train[:, 0].shape

(783,)

In [173]:
def _init_params():
    W1 = np.random.rand(10, 783)
    b1 = np.random.rand(10, 1)
    W2 = np.random.rand(10, 783)
    b2 = np.random.rand(10, 1)
    return W1, b1, W2, b2

In [174]:
def ReLU(Z):
    return np.maximum(0, Z)

def softmax(Z):
    return np.exp(Z) / np.sum(np.exp(Z))

def derivative_ReLU(Z):
    return Z > 0

In [175]:
def froward_prop(W1, b1, W2, b2, X):
    Z1 = W1.dot(X) + b1
    A1 = ReLU(Z1)
    Z2 = W2.T.dot(A1) + b2
    A2 = softmax(Z2)
    return Z1, A1, Z2, A2
    

In [176]:
def one_hot(Y):
    one_hot_Y = np.zeros((Y.size, Y.max() + 1))
    one_hot_Y[np.arange(Y.size), Y] = 1
    one_hot_Y = one_hot_Y.T
    return one_hot_Y

In [177]:
def back_prop(Z1, A1, Z2, A2, X, Y):
    m = Y.size
    one_hot_Y = one_hot(Y)
    dZ2 = A2 - one_hot_Y
    dW2 = 1 / m * dZ2.dot(A1.T)
    db2 = 1 / m * np.sum(dZ2 , 2)
    dZ1 = W2.T.dot(dZ2) * derivative_ReLU(Z1)
    dW1 = 1 / m * dZ1.dot(X.T)
    db1 = 1 / m * np.sum(dZ1, 2)
    return dW1, db1, dW2, db2

In [178]:
def update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha):
    W1 = W1 - alpha * dW1
    b1 = b1 - alpha * db1
    W2 = W2 - alpha * dW2
    b2 = b2 - alpha * db2
    return W1, b1, W2, b2

In [179]:
def get_predictions(A2):
    return np.argmax(A2, 0)

def get_accuracy(predictions, Y):
    print(predictions, Y)
    return np.sum(prediction == Y) / Y.size

In [180]:
def gradient_descent(X, Y, iterations, alpha):
    W1, b1, W2, b2 = _init_params()
    for i in range(iterations):
        Z1, A1, Z2, A2 = froward_prop(W1, b1, W2, b2, X)
        dW1, db1, dW2, db2 = back_prop(Z1, A1, Z2, A2, X, Y)
        W1, b1, W2, b2 = update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha)
        if i % 50 == 0:
            print("Iterations: ", i)
            print("Accuracy: ", get_accuracy(get_predictions(A2), Y))
    return W1, b1, W2, b2

In [181]:
W1, b1, W2, b2 = gradient_descent(X_train, Y_train, 500, 0.1)

ValueError: operands could not be broadcast together with shapes (783,27000) (10,1) 